# E-commerce O-list

In [1]:
import pandas as pd
import numpy as np

In [2]:
customers_df = pd.read_csv("Data/olist_customers_dataset.csv")
geolocation_df = pd.read_csv("Data/olist_geolocation_dataset.csv") 
geolocation_df["geolocation_city"] = geolocation_df["geolocation_city"].replace("são paulo","sao paulo")
orders_items_df = pd.read_csv("Data/olist_order_items_dataset.csv")
orders_payment_df = pd.read_csv("Data/olist_order_payments_dataset.csv")
orders_review_df = pd.read_csv("Data/olist_order_reviews_dataset.csv")
orders_df = pd.read_csv("Data/olist_orders_dataset.csv")
products_df = pd.read_csv("Data/olist_products_dataset.csv")
sellers_df = pd.read_csv("Data/olist_sellers_dataset.csv")
product_name_df = pd.read_csv("Data/product_category_name_translation.csv")

## Product name

In [3]:
products_df = products_df.merge(product_name_df, on='product_category_name').drop('product_category_name', axis=1)
products_df = products_df.rename(columns={'product_category_name_english':'product_category_name'}) 

In [4]:
products_df

,product_id,product_name_lenght,product_description_lenght,product_photos_qty,product_weight_g,product_length_cm,product_height_cm,product_width_cm,product_category_name
0,1e9e8ef04dbcff4541ed26657ea517e5,40.0,287.0,1.0,225.0,16.0,10.0,14.0,perfumery
1,6a2fb4dd53d2cdb88e0432f1284a004c,39.0,346.0,2.0,400.0,27.0,5.0,20.0,perfumery
2,0d009643171aee696f4733340bc2fdd0,52.0,150.0,1.0,422.0,21.0,16.0,18.0,perfumery
3,b1eae565a61935e0011ee7682fef9dc9,49.0,460.0,2.0,267.0,17.0,13.0,17.0,perfumery
4,8da90b37f0fb171b4877c124f965b1f6,56.0,733.0,3.0,377.0,18.0,13.0,15.0,perfumery
...,...,...,...,...,...,...,...,...,...
32323,13b25797ad1e6d6c7cd3cbeb35f8ee7a,44.0,348.0,1.0,18250.0,25.0,25.0,25.0,furniture_mattress_and_upholstery
32324,dc36a7859b743d8610a2bbbaea26ece9,29.0,2073.0,2.0,2500.0,41.0,21.0,21.0,furniture_mattress_and_upholstery
32325,107fde0930956120d1e13dd1062fbb46,51.0,418.0,1.0,30000.0,60.0,73.0,60.0,furniture_mattress_and_upholstery
32326,726b4e18f00255e2e63491bcba3f60b8,41.0,866.0,1.0,1700.0,70.0,10.0,100.0,furniture_mattress_and_upholstery


## Longitude et latitude customer and seller

In [5]:
geolocation_df.columns = ["customer_zip_code_prefix","customer_lat","customer_lng","geolocation_city", "geolocation_state"]
customers_df = customers_df.merge(geolocation_df, how="inner", on="customer_zip_code_prefix")
customers_df = customers_df.groupby(["customer_id", "customer_unique_id","customer_zip_code_prefix","customer_city","customer_state"]).mean().reset_index()

In [6]:
geolocation_df.columns = ["seller_zip_code_prefix","seller_lat","seller_lng","geolocation_city", "geolocation_state"]
sellers_df = sellers_df.merge(geolocation_df, how="inner", on="seller_zip_code_prefix")
sellers_df = sellers_df.groupby(["seller_id","seller_zip_code_prefix","seller_city","seller_state"]).mean().reset_index()

## Fusion finale

In [7]:
df = customers_df.merge(orders_df, on=["customer_id"]).merge(orders_items_df, on="order_id").merge(orders_payment_df, on="order_id").merge(orders_review_df, on="order_id").merge(products_df,on="product_id").merge(sellers_df, on="seller_id")

In [8]:
df = df.drop(["customer_zip_code_prefix","customer_state",
              "customer_city","seller_state",
              "seller_city", "seller_zip_code_prefix",
              "customer_id", "customer_unique_id",
              "seller_id", "product_id", "order_id",
              "order_item_id", "review_id",
              "shipping_limit_date", "order_delivered_carrier_date",
              "order_approved_at", "order_purchase_timestamp"], axis=1)
df

,customer_lat,customer_lng,order_status,order_delivered_customer_date,order_estimated_delivery_date,price,freight_value,payment_sequential,payment_type,payment_installments,...,product_name_lenght,product_description_lenght,product_photos_qty,product_weight_g,product_length_cm,product_height_cm,product_width_cm,product_category_name,seller_lat,seller_lng
0,-23.499063,-46.767939,delivered,2017-11-28 15:41:30,2017-12-04 00:00:00,89.80,24.94,1,credit_card,8,...,55.0,889.0,1.0,4267.0,46.0,39.0,22.0,toys,-23.049552,-47.837621
1,-20.794069,-49.365081,delivered,2017-09-11 19:37:55,2017-09-20 00:00:00,84.80,24.90,1,credit_card,2,...,55.0,889.0,1.0,4267.0,46.0,39.0,22.0,toys,-23.049552,-47.837621
2,-23.009475,-43.334530,delivered,2017-10-28 13:22:55,2017-11-23 00:00:00,89.80,18.00,1,credit_card,2,...,55.0,889.0,1.0,4267.0,46.0,39.0,22.0,toys,-23.049552,-47.837621
3,-23.590310,-46.723572,delivered,2017-08-23 20:39:01,2017-09-11 00:00:00,84.80,14.04,1,credit_card,3,...,55.0,889.0,1.0,4267.0,46.0,39.0,22.0,toys,-23.049552,-47.837621
4,-14.674935,-39.376343,delivered,2017-06-22 14:57:57,2017-07-06 00:00:00,86.99,19.85,1,credit_card,2,...,55.0,889.0,1.0,4267.0,46.0,39.0,22.0,toys,-23.049552,-47.837621
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
115032,-12.982806,-38.458086,delivered,2018-08-16 17:12:37,2018-08-16 00:00:00,171.39,19.35,1,boleto,1,...,38.0,1303.0,2.0,1250.0,52.0,7.0,11.0,furniture_decor,-23.632642,-46.554381
115033,-12.982806,-38.458086,delivered,2018-08-16 17:12:37,2018-08-16 00:00:00,171.39,19.35,1,boleto,1,...,38.0,1303.0,2.0,1250.0,52.0,7.0,11.0,furniture_decor,-23.632642,-46.554381
115034,-23.673394,-46.595241,delivered,2017-11-22 21:26:53,2017-12-06 00:00:00,90.00,16.39,1,credit_card,4,...,18.0,142.0,2.0,1100.0,76.0,18.0,13.0,housewares,-21.834794,-45.401880
115035,-22.805769,-42.957267,delivered,2018-05-10 00:25:27,2018-05-16 00:00:00,130.00,14.44,1,credit_card,3,...,57.0,1209.0,3.0,1600.0,22.0,9.0,23.0,computers_accessories,-25.542326,-49.182195


## Exportation au format csv

In [9]:
df.to_csv("Data/merged_df.csv", index=False)